In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import re

In [2]:
def merge_nhl():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    teams = cities['NHL'].str.extract('([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)([A-Z]{0,2}[a-z0-9]*\ [A-Z]{0,2}[a-z0-9]*|[A-Z]{0,2}[a-z0-9]*)')               
    teams['Metropolitan area'] = cities['Metropolitan area']
    teams = pd.melt(teams, id_vars=['Metropolitan area']).replace('',np.nan).dropna().rename(columns={'value':'team'}).reset_index().drop(columns=['variable','index'])

    teams = teams.merge(cities,how = 'left',on = 'Metropolitan area').iloc[:,0:3].rename(columns={'Population (2016 est.)[8]':'Population'})
    teams = teams.astype({'Metropolitan area':str,'team':str,'Population':int})
    teams['team']=teams['team'].str.replace('[\w.]*\ ','')

    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df['year']==2018]
    nhl_df = nhl_df[nhl_df['GP'].apply(lambda x:x.isnumeric())]
    nhl_df = nhl_df[['team','W','L']]
    nhl_df['team'] = nhl_df['team'].apply(lambda x: x.split(' ')[-1].strip('*'))
    nhl_df = nhl_df.astype({'W':int,'L':int})
    nhl_df['W/L%'] = nhl_df['W']/(nhl_df['W']+nhl_df['L'])

    merge = pd.merge(teams,nhl_df,how = 'outer', on = 'team')

    merge = merge.groupby('Metropolitan area').agg({'W/L%':np.nanmean,'Population':np.nanmean})
    return merge

In [4]:
def merge_nba():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    teams = cities['NBA'].str.extract('([A-Z0-9]{0,2}\w*\ [A-Z]{0,2}\w*|[A-Z0-9]{0,2}[a-z]*)([A-Z0-9]{0,2}\w*|[A-Z0-9]{0,2}\s\w*)')
    teams['Metropolitan area'] = cities['Metropolitan area']
    teams = pd.melt(teams,id_vars='Metropolitan area').reset_index().drop(columns=['variable','index']).replace('',np.nan).dropna().rename(columns={'value':'team'})
    teams = teams.merge(cities,how = 'left', on = 'Metropolitan area').iloc[:,[0,1,2]].rename(columns={'Population (2016 est.)[8]':'Population'}).astype({'Population':int})
    teams['team'] = teams['team'].apply(lambda x: x.split(' ')[-1])

    nba_df = pd.read_csv("assets/nba.csv")
    nba_df = nba_df[nba_df['year']==2018]
    nba_df = nba_df.iloc[:,[0,1,2]]
    nba_df['team'] = nba_df['team'].apply(lambda x: x.split('(')[0])
    nba_df['team'] = nba_df['team'].apply(lambda x: x.split('*')[0])
    nba_df['team'] = nba_df['team'].apply(lambda x: x.split(' ')[-1].strip())
    nba_df = nba_df.astype({'W':int,'L':int})
    nba_df['W/L%'] = nba_df['W']/(nba_df['W']+nba_df['L'])

    merge = pd.merge(nba_df,teams,how = 'left',on = 'team')
    merge = merge.groupby('Metropolitan area').agg({'W/L%':np.mean,'Population':np.mean})
    return merge

In [15]:
def merge_mlb():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    teams = cities['MLB'].str.extract('([A-Z]{0,2}[a-z]*\ [A-Z]{0,2}[a-z]*|[A-Z]{0,2}[a-z]*)([A-Z]{0,2}[a-z]*\ [A-Z]{0,2}[a-z]*|[A-Z]{0,2}[a-z]*)([A-Z]{0,2}[a-z]*)([A-Z]{0,2}[a-z]*)([A-Z]{0,2}[a-z]*)([A-Z]{0,2}[a-z]*)')
    teams['Metropolitan area'] = cities['Metropolitan area']
    teams = pd.melt(teams,id_vars='Metropolitan area').replace('',np.nan).dropna().reset_index().drop(columns=['index','variable']).rename(columns={'value':'team'})
    teams = teams.merge(cities,how='left',on = 'Metropolitan area').iloc[:,[0,1,2]].rename(columns={'Population (2016 est.)[8]':'Population'}).astype({'Population':int})
    teams['team'] = teams['team'].apply(lambda x:x.split(' ')[-1].strip())
    teams['team'].iloc[7] = 'Red Sox'

    mlb_df = pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df[mlb_df['year']==2018].iloc[:,[0,1,2]]
    mlb_df['team'] = mlb_df['team'].apply(lambda x:x.split(' ')[-1].strip())
    mlb_df = mlb_df.astype({'W':int,'L':int})
    mlb_df['W/L%']= mlb_df['W']/(mlb_df['W']+mlb_df['L'])
    mlb_df['team'].iloc[0] = 'Red Sox'

    merge = pd.merge(mlb_df,teams,how ='left',on = 'team')
    merge = merge.groupby('Metropolitan area').agg({'W/L%':np.nanmean,'Population':np.nanmean})
    return merge

In [7]:
def merge_nfl():
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    teams = cities['NFL'].str.extract('([A-Z0-9]{0,2}\w*\ [A-Z]{0,2}\w*|[A-Z0-9]{0,2}[a-z]*)([A-Z0-9]{0,2}\w*|[A-Z0-9]{0,2}\s\w*)')
    teams['Metropolitan area'] = cities['Metropolitan area']
    teams = pd.melt(teams,id_vars='Metropolitan area').reset_index().drop(columns=['variable','index']).replace('',np.nan).dropna().rename(columns={'value':'team'})
    teams = teams.merge(cities,how = 'left', on = 'Metropolitan area').iloc[:,[0,1,2]].rename(columns={'Population (2016 est.)[8]':'Population'}).astype({'Population':int})
    teams['team'] = teams['team'].apply(lambda x: x.split(' ')[-1])

    teams.head()

    nfl_df = pd.read_csv('assets/nfl.csv')
    nfl_df = nfl_df[nfl_df['year']==2018][['team','W','L']]
    nfl_df = nfl_df[nfl_df['W'].apply(lambda x:x.isnumeric())].astype({'W':int,'L':int})
    nfl_df['team'] = nfl_df['team'].apply(lambda x: x.strip('*+'))
    nfl_df['team'] = nfl_df['team'].apply(lambda x: x.split(' ')[-1])
    nfl_df['W/L%'] = nfl_df['W']/(nfl_df['W']+nfl_df['L'])

    merge = pd.merge(nfl_df,teams,how = 'left',on = 'team')
    merge = merge.groupby('Metropolitan area').agg({'W/L%':np.nanmean,'Population':np.nanmean})
    return merge

In [16]:
def create_df(sport):
    if sport =='NFL':
        return merge_nfl()
    elif sport =='NBA':
        return merge_nba()
    elif sport =='NHL':
        return merge_nhl()
    elif sport =='MLB':
        return merge_mlb()

In [17]:
sports = ['NFL','NBA','NHL','MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
for i in sports:
    for j in sports:
        if i!=j:
            merge = create_df(i).merge(create_df(j),how='inner',on='Metropolitan area')
            p_values[i].loc[j] = stats.ttest_rel(merge['W/L%_x'],merge['W/L%_y'])[1]

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [18]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,0.941792,0.030883,0.802069
NBA,0.941792,NaN,0.022297,0.950540
NHL,0.030883,0.022297,NaN,0.000708
MLB,0.802069,0.950540,0.000708,NaN
